<a href="https://colab.research.google.com/github/carvalheiracarlos/deep_leaning_notebooks/blob/main/text_generation/twitter_generation_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
import os
import time

In [ ]:
path_to_file = 'xorume_do_bolsonaro.txt'
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 395857 characters


In [ ]:
print(text[:250])

- É preciso compreender aqueles que ainda não decidiram e lhes oferecer segurança para que façam a melhor escolha para o futuro da nossa nação. Mais do que promessas vazias e abstratas, o Brasil precisa de um caminho sólido, pautado em ações concreta


In [ ]:
#text = text.apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))

In [ ]:
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')
print(vocab[:250])

176 unique characters
['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', '\xa0', '¡', 'ª', '°', 'º', 'À', 'Á', 'Â', 'Ã', 'Ç', 'É', 'Ê', 'Í', 'Ñ', 'Ó', 'Ô', 'Õ', 'Ú', 'à', 'á', 'â', 'ã', 'ç', 'é', 'ê', 'í', 'ñ', 'ó', 'ô', 'õ', 'ú', 'ᕵ', 'ᖶ', '\u200b', '–', '—', '‘', '’', '“', '”', '•', '…', '₱', '₵', '⃣', '⚓', '✅', '✈', '➡', '⬇', 'い', 'し', 'ら', '日', '晴', '素', '️', '🇦', '🇧', '🇨', '🇪', '🇬', '🇭', '🇮', '🇯', '🇱', '🇴', '🇵', '🇷', '🇸', '🇹', '🇺', '🇻', '🇾', '🌃', '🌎', '🍕', '🎸', '🏻', '🏼', '👉', '👍', '😂', '🙏', '🚁', '🛫', '🤔', '🤖', '🤝', '🥤']


In [ ]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [ ]:
# characters -> numbers (layer)
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [ ]:
# It converts from tokens to character IDs:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[60, 61, 62, 63, 64, 65, 66], [83, 84, 85]]>

In [ ]:
# numbers -> ids (layer)
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [ ]:
# convert numbers -> characters
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [ ]:
# join characters back into text (layer)
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [ ]:
# convert numbers into text
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [ ]:
# text -> stream of indices (numbers)
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(395857,), dtype=int64, numpy=array([15,  2, 97, ..., 72, 64,  1])>

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

-
 
É
 
p
r
e
c
i
s


In [ ]:
# size of the textual sequences to be used
seq_length = 100

sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'-' b' ' b'\xc3\x89' b' ' b'p' b'r' b'e' b'c' b'i' b's' b'o' b' ' b'c'
 b'o' b'm' b'p' b'r' b'e' b'e' b'n' b'd' b'e' b'r' b' ' b'a' b'q' b'u'
 b'e' b'l' b'e' b's' b' ' b'q' b'u' b'e' b' ' b'a' b'i' b'n' b'd' b'a'
 b' ' b'n' b'\xc3\xa3' b'o' b' ' b'd' b'e' b'c' b'i' b'd' b'i' b'r' b'a'
 b'm' b' ' b'e' b' ' b'l' b'h' b'e' b's' b' ' b'o' b'f' b'e' b'r' b'e'
 b'c' b'e' b'r' b' ' b's' b'e' b'g' b'u' b'r' b'a' b'n' b'\xc3\xa7' b'a'
 b' ' b'p' b'a' b'r' b'a' b' ' b'q' b'u' b'e' b' ' b'f' b'a' b'\xc3\xa7'
 b'a' b'm' b' ' b'a' b' ' b'm' b'e'], shape=(101,), dtype=string)


In [ ]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'- \xc3\x89 preciso compreender aqueles que ainda n\xc3\xa3o decidiram e lhes oferecer seguran\xc3\xa7a para que fa\xc3\xa7am a me'
b'lhor escolha para o futuro da nossa na\xc3\xa7\xc3\xa3o. Mais do que promessas vazias e abstratas, o Brasil precisa'
b' de um caminho s\xc3\xb3lido, pautado em a\xc3\xa7\xc3\xb5es concretas e, sobretudo, em princ\xc3\xadpios.\n- Logo mais, \xc3\xa0s 19h, f'
b'aremos um importante an\xc3\xbancio em nossas redes sociais. Pe\xc3\xa7o que compartilhem este convite com todos os'
b' seus contatos para que o nosso an\xc3\xbancio chegue a todos os brasileiros. Voltamos em breve!\n- Diferente'


In [ ]:
# For training you'll need a dataset of (input, label) pairs.
# Where input and label are sequences.
# At each time step the input is the current character and the label is the next character.
# Here's a function that takes a sequence as input, duplicates, and shifts it
# to align the input and label for each timestep:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
dataset = sequences.map(split_input_target)

for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'- \xc3\x89 preciso compreender aqueles que ainda n\xc3\xa3o decidiram e lhes oferecer seguran\xc3\xa7a para que fa\xc3\xa7am a m'
Target: b' \xc3\x89 preciso compreender aqueles que ainda n\xc3\xa3o decidiram e lhes oferecer seguran\xc3\xa7a para que fa\xc3\xa7am a me'


In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [ ]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
# building the model
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
# load model
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
# trying the model
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 177) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  45312     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  181425    
                                                                 
Total params: 4,165,041
Trainable params: 4,165,041
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# To get actual predictions from the model you need to sample from the output distribution,
#to get actual character indices. This distribution is defined by the logits over the character vocabulary.
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [ ]:
# example
sampled_indices

array([163,  51,  80,  17,  74, 132, 147,   3,   8, 103, 156, 174,  34,
        99,  11, 166, 121, 107,  45,  30,  87, 141, 164,  89, 175, 130,
       162, 125, 111, 159, 127,   6, 151,  75,  19, 168,  10, 147,  25,
        74, 104,  66,  56,  25,   3,  42,  26, 166, 114,  85, 167, 145,
        28,  18, 127,  67, 144, 137,  91,  78, 120, 135,  15,  42, 157,
       117,  96,  47, 113, 153,  74,  43, 131, 133,  30, 106, 100,  51,
       126, 134, 129,  41, 125, 154,  79, 174,  71,   7, 167, 131, 154,
        21, 112,  44,  53, 159, 127,  47,  52,  59])

In [ ]:
# Decode these to see the text predicted by this untrained model:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b' que est\xc3\xa3o com libera\xc3\xa7\xc3\xa3o sendo acelerada. (Segue)\nCumprimento Joe Biden como 46\xc2\xba Presidente dos EUA.'

Next Char Predictions:
 b'\xf0\x9f\x8d\x95Su/o\xe2\x9a\x93\xf0\x9f\x87\xaa!&\xc3\x95\xf0\x9f\x87\xb8\xf0\x9f\xa4\x96B\xc3\x8d)\xf0\x9f\x8f\xbc\xe2\x80\x93\xc3\xa2M=\xc2\xa0\xe6\x99\xb4\xf0\x9f\x8e\xb8\xc2\xaa\xf0\x9f\xa4\x9d\xe2\x82\xb5\xf0\x9f\x8c\x8e\xe2\x80\x9c\xc3\xaa\xf0\x9f\x87\xbb\xe2\x80\xa2$\xf0\x9f\x87\xafp1\xf0\x9f\x91\x8d(\xf0\x9f\x87\xaa7o\xc3\x9agX7!J8\xf0\x9f\x8f\xbc\xc3\xb3z\xf0\x9f\x91\x89\xf0\x9f\x87\xa7:0\xe2\x80\xa2h\xf0\x9f\x87\xa6\xe3\x81\x84\xc2\xbas\xe2\x80\x8b\xe2\x9e\xa1-J\xf0\x9f\x87\xb9\xc3\xba\xc3\x87O\xc3\xb1\xf0\x9f\x87\xb4oK\xe2\x83\xa3\xe2\x9c\x85=\xc3\xa1\xc3\x91S\xe2\x80\x9d\xe2\x9c\x88\xe2\x82\xb1I\xe2\x80\x9c\xf0\x9f\x87\xb5t\xf0\x9f\xa4\x96l%\xf0\x9f\x91\x89\xe2\x83\xa3\xf0\x9f\x87\xb53\xc3\xadLU\xf0\x9f\x87\xbb\xe2\x80\xa2OT_'


In [ ]:
# cost function
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
# cost (untrained)
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 177)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(5.177214, shape=(), dtype=float32)


In [ ]:
tf.exp(example_batch_mean_loss).numpy()

177.1885

In [ ]:
# model compilation
model.compile(optimizer='adam', loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
EPOCHS = 20

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
61/61 [==============================] - 6s 55ms/step - loss: 3.3986
Epoch 2/20
61/61 [==============================] - 4s 55ms/step - loss: 2.5241
Epoch 3/20
61/61 [==============================] - 4s 56ms/step - loss: 2.3236
Epoch 4/20
61/61 [==============================] - 4s 56ms/step - loss: 2.1761
Epoch 5/20
61/61 [==============================] - 4s 57ms/step - loss: 2.0371
Epoch 6/20
61/61 [==============================] - 4s 57ms/step - loss: 1.8991
Epoch 7/20
61/61 [==============================] - 4s 59ms/step - loss: 1.7668
Epoch 8/20
61/61 [==============================] - 4s 58ms/step - loss: 1.6507
Epoch 9/20
61/61 [==============================] - 4s 58ms/step - loss: 1.5527
Epoch 10/20
61/61 [==============================] - 4s 58ms/step - loss: 1.4660
Epoch 11/20
61/61 [==============================] - 4s 59ms/step - loss: 1.3924
Epoch 12/20
61/61 [==============================] - 4s 59ms/step - loss: 1.3224
Epoch 13/20
61/61 [==================

In [ ]:
# text generation model
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['Bolsonaro:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Bolsonaro:
1- O @govbr consumido há pouco à aborto a desbinação fiz ao seu país entre as nações em comandonados pelo caro
Ferendizento básico referando o caminho-ceperal ao PT e investir ao sensalão que potencialmente para negecires que serão destróinas de março do Siotórica. Codais baixos merecemos mais recussos sem setra 15 pensionistas.
- UTA WEFRS-Jão: Etusa/entenca.
O terrorás na Brasil pode mercado envolvendo mais justares até 2020, uma povia agrária"?
O dia 11 agora).
Mais uma vez mais avançam a cobrar defendendo a compar come signões que conticulativos menos sua autoridade e investimentos nos meses. Também com profissionais e grandes possibilidade de informar, que seja hando ser a passagem de dinheiro das atuais assinados, cujo objetivo para se concluívam aos ideolos isso.
- No dia de o trmísão em 2019 do Uma that, apresentado pela cristinh, aviação sibnicipal com a segurança e investiu R$ 15, bilhões, não gerem repossem obsas, autonomia pauta e após ser um deseja, nez a esquel